# goal: build a crew for planning an event

## notes and questions
- pydantic objects (called pydantic "models") help us enforce some sort of strong-typing in this fuzzy agentic universe.
- passing a pydantic object to the`output_json` argument for a task lets us specify the structure of the output we want along with the data type of each part of that output.
- the `output_file` argument lets us store the output in a file.
- setting `human_input` to True makes the task for human feedback (whether you like the results or not) before finalising it
- setting `async_execution` to True means the task can run in parallel with the tasks which come after it.
    - note that `async_execution` needs to be set to True for all tasks that you want running in parallel since they are independent of one another. setting it for just one task will **not** make it automatically apply to any other tasks you define after it or list in the crew definition after it.

## main code

In [1]:
import warnings
warnings.filterwarnings('ignore')  # don't bother showing warnings

from crewai import Agent, Task, Crew

from IPython.display import Markdown

import os
from utils import get_openai_api_key, pretty_print_result, get_serper_api_key
openai_api_key = get_openai_api_key()
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

In [3]:
# tools

from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# initialise tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [4]:
# agents

# venue scout
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)


# logistics/ops manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipment"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)


# event marketing and participation-communication agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

In [5]:
# pydantic objects (called pydantic "models")

from pydantic import BaseModel

# define a pydantic BaseModel object for storing venue details
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

In [7]:
# tasks

# finding the perfect venue
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,  # the agent will ask the user if they like the result before finalising it
    output_json=VenueDetails,  # the pydantic object defining the structure we want the output to have
    output_file="venue_details.json",  # output stored
    agent=venue_coordinator
)


# planning all logistics for the venue selected by the venue coordinator
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,  # will need HITL
    async_execution=True,  # will run in parallel w the tasks that come after it in the crew definition
    agent=logistics_manager
)


# event marketing and communications
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # outputs the report as a markdown file
    agent=marketing_communications_agent
)

In [8]:
# assembling the crew

event_management_crew = Crew(
    agents=[venue_coordinator, logistics_manager, marketing_communications_agent],    
    tasks=[marketing_task, venue_task, logistics_task],  # since logistics and marketing have been set to run in parallel, their order in this list does not matter. it does matter that logistics comes after venue coordination though, since the logistics task will require knowing the venue.
    verbose=True
)

In [11]:
event_management_crew = Crew(
    agents=[venue_coordinator, logistics_manager, marketing_communications_agent],    
    tasks=[venue_task, marketing_task, logistics_task],  # since logistics and marketing have been set to run in parallel, their order in this list does not matter. it does matter that logistics comes after venue coordination though, since the logistics task will require knowing the venue.
    verbose=True
)

event_details = {
    'event_topic': "Kodosumi Hackathon",
    'event_description': "A hands-on workshop on coding AI agents "
                         "followed by a hackathon in which teams "
                         "come up with use cases for AI agents and "
                         "implement them. ",
    'event_city': "Munich, Germany",
    'tentative_date': "2025-05-15",
    'expected_participants': 100,
    'budget': 10000,
    'venue_type': "Coworking Space"
}

result = event_management_crew.kickoff(inputs=event_details)

2025-03-03 10:58:46,426 - 8469596224 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in Munich, Germany that meets criteria for Kodosumi Hackathon.


> Entering new CrewAgentExecutor chain...
Thought: To find a suitable venue for the Kodosumi Hackathon in Munich, Germany, I need to determine the specific requirements of the event such as capacity, technology needs, and any special accommodations. I should start by searching for information about available venues in Munich that can host hackathons.

Action: Search the internet
  Input: {"search_query": "venues in Munich Germany suitable for hackathons"}


Search results: Title: Hackathon locations in Munich - Spacebase
Link: https://www.spacebase.com/en/munich/hackathon/
Snippet: Rent inspiring Hackathon locations in Munich online! ✓modern & extraordinary venues ✓reserve for free ✓catering & equipment.
---
Title: Any upcoming hackathons in Germany? - Reddit
Link: https://www.reddit.com/r/germany/comments/1g7e8m8/any_upcoming_hackathons

This is the agent final answer: Dachwerk Event - Dachwerk Event is a suitable venue for the Kodosumi Hackathon in Munich, Germany. Located in the Glockenbach Quarter, this venue offers a creative space that can accommodate 10 to 60 people. The venue is well-equipped for professional events, providing a spacious area of 110 sqm with various seating arrangements, including standing (up to 60 people), parliament (up to 40 people), U-Shape (up to 28 people), and theatre (up to 50 people). The pricing is flexible with rates starting from €950 for half-day sessions to €1450 for a full day. Additionally, the venue has received positive reviews for its excellent service and facilities. This venue should meet the requirements for the Kodosumi Hackathon, providing both the necessary space and amenities needed for a successful event.
Please provide a feedback:  My event has 100 participants. This venue will be too small.


Thought: Given the requirement to accommodate 100 participants, the previously identified venue, Dachwerk Event, is insufficient as its maximum capacity is 60 people. I need to search for another venue in Munich that can comfortably handle 100 participants and meets the requirements for a hackathon.

Action: Search the internet
  Input: {"search_query": "large event venues in Munich suitable for hackathons with capacity of 100 or more"}


Search results: Title: Venues up to 100 persons on Munich
Link: https://de.fiylo.com/venues-up-to-100-persons/muenchen/
Snippet: Are you looking for extraordinary event locations for up to 300 people? BMW Welt in Munich has the perfect solution for your tailor-made event. The fascinating ...
---
Title: Hackathon locations in Munich - Spacebase
Link: https://www.spacebase.com/en/munich/hackathon/
Snippet: Rent inspiring Hackathon locations in Munich online! ✓modern & extraordinary venues ✓reserve for free ✓catering & equipment.
---
Title: Event and Mee

Exception in thread Thread-14 (_execute):
Traceback (most recent call last):
  File "/Users/joshis/.pyenv/versions/3.11.0/lib/python3.11/threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "/Users/joshis/.pyenv/versions/learnagentic/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/Users/joshis/.pyenv/versions/3.11.0/lib/python3.11/threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/joshis/.pyenv/versions/learnagentic/lib/python3.11/site-packages/crewai/task.py", line 182, in _execute
    result = agent.execute_task(
             ^^^^^^^^^^^^^^^^^^^
  File "/Users/joshis/.pyenv/versions/learnagentic/lib/python3.11/site-packages/crewai/agent.py", line 221, in execute_task
    result = self.agent_executor.invoke(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/joshis/.pyenv/versions/learnagentic/lib/python3.11/site-packages/langchain/chains/base.py", line 

In [12]:
Markdown(result)

{
  "name": "BMW Welt",
  "address": "Am Olympiapark 1, 80809 München",
  "capacity": 300,
  "booking_status": "Available"
}

In [14]:
Markdown("marketing_report.md")

# Marketing Report for Kodosumi Hackathon

## Objective
Promote the Kodosumi Hackathon at BMW Welt, aiming to engage at least 100 potential attendees.

## Venue Details
- **Name**: BMW Welt
- **Address**: Am Olympiapark 1, 80809 München
- **Capacity**: 300 attendees
- **Booking Status**: Available

## Marketing and Promotion Strategies

### 1. Press Releases and Public Relations
- **Objective**: Generate awareness and interest through media.
- **Action**: Craft a well-targeted press release highlighting the hackathon’s aim, theme, timeline, benefits for participants, and available prizes.
- **Distribution**: Focus on tech, innovation, HR, and marketing media outlets. Consider platforms like Inc.com and Entrepreneur.com for broader reach.

### 2. Social Media Strategy
- **Platforms**: Twitter, Facebook, Instagram, and LinkedIn.
- **Actions**:
  - **Facebook**: Create a dedicated hackathon page for continuous updates and engage with the community through a public event.
  - **Twitter**: Increase posting frequency to 3-4 times per day using relevant hashtags to engage with tech communities.
  - **LinkedIn**: Utilize for reaching startups and professional communities.
  - **Instagram**: Use with visually appealing posts and relevant hashtags to maximize engagement.
- **Advertising**: Implement targeted ads across these platforms to maximize reach and engagement.

### 3. Engaging with Educational Institutions and Incubators
- **Target**: Startups, students, and developers.
- **Actions**:
  - **Startups**: Partner with incubators, accelerators, and co-working spaces to promote the event.
  - **Students and Developers**: Connect with universities and colleges through administrative and teaching staff, utilizing Agorize’s network for wider coverage.

### 4. Event Participation
- **Objective**: Increase physical presence and engagement at relevant events.
- **Actions**: Attend tech conferences, student fairs, and public exhibitions to promote the hackathon. Distribute materials like flyers, posters, and other goodies.

### 5. Continuous Engagement
- **Pre-Event**: Maintain communication with registered participants through email reminders and social media updates.
- **During Event**: Ensure active management of social media channels to interact with participants and address any issues promptly.

## Conclusion
The detailed strategy aims to not only attract but also engage a significant number of participants to the Kodosumi Hackathon, leveraging both digital and physical promotional avenues to ensure a successful event with high participation.